In [52]:
# svm.py
import numpy as np  # for handling multi-dimensional array operation
import pandas as pd  # for reading data from csv 
import statsmodels.api as sm  # for finding the p-value
from sklearn.preprocessing import MinMaxScaler  # for normalization
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score 
from sklearn.utils import shuffle
import os
os.chdir('C:/Users/mithu/OneDrive/ML/SVM')


The dataset consists of large number of features. We’ll be working with a breast cancer dataset available on Kaggle. The features in the dataset are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. They describe the characteristics of the cell nuclei present in the image. Based on these features we will train our SVM model to detect if the mass is benign B (generally harmless) or malignant M (cancerous). 

In [53]:
df=pd.read_csv('data.txt',header=0)
df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


Preprocessing the dataset:Since we will use hinge loss in the SVM loss function, the output needs to be [-1,1] values.

In [54]:
mapping={'M':1,'B':-1}
df.diagnosis=df.diagnosis.map(mapping)
Y=df.diagnosis
Y=Y.astype('float64')############

Droping any missing values for this analysis. 

In [55]:
df = df.drop(df.columns[[0,1, -1]], axis=1)  # df.columns is zero-based pd.Index
df

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


We than rescale the feature into [0,1] scale for each features. 

In [56]:
scaleX=MinMaxScaler().fit_transform(df)
X=pd.DataFrame(scaleX)


Feature selection:
<ol>
    <li> Elemenating the correlated feature.</li>
    <li> Eleminating the less significant features</li>
</ol>
 <p>The correlated features can be found from the correlations between two columns of the dataframe. If the correlations are >=0.9, we will eliminate the features as it is completely redundant and can be expressed by other features.
 <p> In order to eliminate the unwanted or irrelevant features, we will use backward elimination with the OLS fit. That means, we initially take all the features and fit it to OLS and then check the corresponding features p-values. If its value>0.05, we will eliminate that feature and refit again with the new features.

In [57]:
def correlated_features(X):
    corr_threshold = 0.9
    corr = X.corr().values
    deleted_column=[]
    for i in range(X.shape[1]):
        for j in range(i+1,X.shape[1]):
            if corr[i,j]>=corr_threshold:
                deleted_column.append(j)
    X=X.drop(X.columns[deleted_column], axis=1,inplace=True)
    return X

In [58]:
correlated_features(X)

We got 10 correlatd features which we eliminate from the analysis

In [59]:
X

,0,1,4,5,6,8,9,10,11,14,15,16,17,18,19,24,25,26,28,29
0,0.521037,0.022658,0.593753,0.792037,0.703140,0.686364,0.605518,0.356147,0.120469,0.159296,0.351398,0.135682,0.300625,0.311645,0.183042,0.601136,0.619292,0.568610,0.598462,0.418864
1,0.643144,0.272574,0.289880,0.181768,0.203608,0.379798,0.141323,0.156437,0.082589,0.119387,0.081323,0.046970,0.253836,0.084539,0.091110,0.347553,0.154563,0.192971,0.233590,0.222878
2,0.601496,0.390260,0.514309,0.431017,0.462512,0.509596,0.211247,0.229622,0.094303,0.150831,0.283955,0.096768,0.389847,0.205690,0.127006,0.483590,0.385375,0.359744,0.403706,0.213433
3,0.210090,0.360839,0.811321,0.811361,0.565604,0.776263,1.000000,0.139091,0.175875,0.251453,0.543215,0.142955,0.353665,0.728148,0.287205,0.915472,0.814012,0.548642,1.000000,0.773711
4,0.629893,0.156578,0.430351,0.347893,0.463918,0.378283,0.186816,0.233822,0.093065,0.332359,0.167918,0.143636,0.357075,0.136179,0.145800,0.437364,0.172415,0.319489,0.157500,0.142595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0.690000,0.428813,0.526948,0.296055,0.571462,0.336364,0.132056,0.385479,0.197976,0.291906,0.200213,0.131263,0.464861,0.045843,0.115536,0.461137,0.178527,0.328035,0.097575,0.105667
565,0.622320,0.626987,0.407782,0.257714,0.337395,0.349495,0.113100,0.236828,0.464728,0.137879,0.165064,0.099747,0.317863,0.156160,0.055387,0.300007,0.159997,0.256789,0.198502,0.074315
566,0.455251,0.621238,0.288165,0.254340,0.216753,0.267677,0.137321,0.124896,0.157974,0.142435,0.263301,0.119444,0.294942,0.074548,0.103547,0.282177,0.273705,0.271805,0.128721,0.151909
567,0.644564,0.663510,0.588336,0.790197,0.823336,0.675253,0.425442,0.222524,0.272896,0.163477,0.445579,0.179722,0.315211,0.216103,0.182766,0.619626,0.815758,0.749760,0.497142,0.452315


In [60]:
#backward elemination from p-values
def less_significant_features(X,y):
    alpha=0.05
    col_drop=[]
    for i in range(len(X.columns)):
        reg=sm.OLS(y,X).fit()
        maxcol=reg.pvalues.idxmax()
        maxval=reg.pvalues.max()
        if maxval>=alpha:
            col_drop.append(maxcol)
            X.drop(maxcol, axis='columns', inplace=True)
        else:
            break
        reg.summary()
    return col_drop

The less significant features in the datafreame is printed below. Thus now we have 14 features with one bias term. 

In [61]:
drop_col=less_significant_features(X,Y)
X['0']=np.repeat(1,len(X.index))
cols = X.columns.tolist()
cols=list(cols[-1])+cols[0:-1]
X=X[cols]

In [62]:
X

,0,0,4,5,6,8,9,10,15,16,17,19,24,28,29
0,1,0.521037,0.593753,0.792037,0.703140,0.686364,0.605518,0.356147,0.351398,0.135682,0.300625,0.183042,0.601136,0.598462,0.418864
1,1,0.643144,0.289880,0.181768,0.203608,0.379798,0.141323,0.156437,0.081323,0.046970,0.253836,0.091110,0.347553,0.233590,0.222878
2,1,0.601496,0.514309,0.431017,0.462512,0.509596,0.211247,0.229622,0.283955,0.096768,0.389847,0.127006,0.483590,0.403706,0.213433
3,1,0.210090,0.811321,0.811361,0.565604,0.776263,1.000000,0.139091,0.543215,0.142955,0.353665,0.287205,0.915472,1.000000,0.773711
4,1,0.629893,0.430351,0.347893,0.463918,0.378283,0.186816,0.233822,0.167918,0.143636,0.357075,0.145800,0.437364,0.157500,0.142595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,1,0.690000,0.526948,0.296055,0.571462,0.336364,0.132056,0.385479,0.200213,0.131263,0.464861,0.115536,0.461137,0.097575,0.105667
565,1,0.622320,0.407782,0.257714,0.337395,0.349495,0.113100,0.236828,0.165064,0.099747,0.317863,0.055387,0.300007,0.198502,0.074315
566,1,0.455251,0.288165,0.254340,0.216753,0.267677,0.137321,0.124896,0.263301,0.119444,0.294942,0.103547,0.282177,0.128721,0.151909
567,1,0.644564,0.588336,0.790197,0.823336,0.675253,0.425442,0.222524,0.445579,0.179722,0.315211,0.182766,0.619626,0.497142,0.452315


We are splitting the data into train-test (80%-20%). 

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


We will use the hinge loss function inside the cost functionfor the SVM classification model. The loss function is:
$$max(0,1-y_i(w*x_i+b))$$

The overall cost function is:
$$J(w)=\frac{1}{N}\sum_i^{n}[1/2||w||^2+Cmax(0,1-y_i*(w*x_i))]$$

In this cost function,C is essentially a regularization parameter and smaller C gives a wider margin and vice versa. We need to tune this parameter for the better fitting of the SVM.
The gradient of the cost function is:
$$\bigtriangledown_wJ(w)=\frac{1}{N}\sum_i^{n}
\begin{cases}
      w & \text{if $max(0,1-y_i*(w*x_i))=0$}\\
      w-C(y_ix_i) & \text{otherwise}
\end{cases}$$




In [67]:
class Svm():
    def __init__(self,X,y):
        self.X=X
        self.y=y
        self.w=np.zeros(X.shape[1])
        self.N=self.y.shape[0]
    def predict(self,x,w):
        n=x.shape[0]
        f=np.dot(x,w)
        return np.sign(f)
    def cost(self,C,X,Y):
        # calculate hinge loss
        N = X.shape[0]
        distances = 1 - Y * (np.dot(X, self.w.reshape(-1,1)))
        distances[distances < 0] = 0  # equivalent to max(0, distance)
        hinge_loss = C * (np.sum(distances) / N)        
        # calculate cost
        cost = 0.5* np.dot(self.w, self.w.T) + hinge_loss
        return cost

    def gradient(self,X,y,C):
        # if only one example is passed (eg. in case of SGD)
        if type(y) == np.float64:
            y = np.array([y])
            X = np.array([X])
        distance=1-y*np.dot(X,self.w.reshape(-1,1))
        dw = np.zeros(self.w.shape[0])
        for ind, d in enumerate(distance):
            if max(0, d) == 0:
                di = self.w
            else:
                di = self.w - (C * y[ind] *X[ind])
            dw += di.reshape(-1,)
            dw = dw/(self.N)  # average
        return dw
    
    def sgd(self,maxepoachs=5000,learning_rate=0.0001,C=1000):
        # stochastic gradient descent
        nth = 0
        prev_cost = float("inf")
        cost_threshold = 0.0001  # in percent
        for epoch in range(1,maxepoachs):
            X,Y=shuffle(self.X,self.y)
            for ind,x in enumerate(X):
                ascent=self.gradient(x,Y[ind],C)
                self.w = self.w - (learning_rate * ascent)
            # convergence check on 2^nth epoch
            if (epoch == 2 ** nth) and (epoch <65):#or (epoch == maxepoachs - 1)
                cost = self.cost(C,X,Y)
                print("Epoch is:{} and Cost is: {}".format(epoch, cost))
                # stoppage criterion
                if abs(prev_cost - cost) < cost_threshold * prev_cost:
                    return self.w
                prev_cost = cost
                nth += 1        
        return self.w

We npw call the SVM for the classification in the breast cancer dataset. We print out each 
$2^i$ iteration's cost fuction value to see how it is progressing towards convergence. We use stochastic gradient optimization techniques to find out the weight value from the cost function.
Now what should be the stoppage criterion? There are multiple options, but we will use the simplest one. We will stop the training when the current cost hasn’t decreased much as compared to the previous cost. Above is how we defined sgd() with stoppage criterion.

In [68]:
# train the model
print("training started...")
model=Svm(X_train.to_numpy(), y_train.to_numpy())
W = model.sgd()
print("training finished.")
print("weights are: {}".format(W))


training started...
Epoch is:1 and Cost is: 451336.6795509076
Epoch is:2 and Cost is: 447673.59078568104
Epoch is:4 and Cost is: 440348.6414961556
Epoch is:8 and Cost is: 425703.1727425501
Epoch is:16 and Cost is: 396429.8330287953
Epoch is:32 and Cost is: 348404.4905137342
Epoch is:64 and Cost is: 356538.7075793749
training finished.
weights are: [-3.84029359  3.89027872 -0.38068368  1.30151695  3.67088843 -0.00941456
 -2.11379006  2.69143106 -0.90500541 -1.02743002  0.14925503 -1.1171563
  2.3714462   2.26321023  1.29763928]


After training the model using SGD we finally got the optimal weights w* which defines the best possible hyperplane separating two classes.Let's test our model for the test function and see its performance.

In [66]:
from sklearn.metrics import accuracy_score ,recall_score

y_test_predicted=model.predict(X_test, W)
print("accuracy on test dataset: {}".format(accuracy_score(y_test.to_numpy(), y_test_predicted)))
print("recall on test dataset: {}".format(recall_score(y_test.to_numpy(), y_test_predicted)))
print("precision on test dataset: {}".format(recall_score(y_test.to_numpy(), y_test_predicted)))


accuracy on test dataset: 0.9736842105263158
recall on test dataset: 0.9534883720930233
precision on test dataset: 0.9534883720930233


As you can see the accuracy (97%), precision (0.95), and recall (0.95) scores have improved. Thus, the performance of the model on this dataframe is excellent.